# HiFi-GAN with SageMaker
[HiFi-GAN](https://github.com/jik876/hifi-gan) のトレーニングを SageMaker Training で行います。

## コンテナの作成
* オリジナルの HiFi-GANでは、PyTorch 1.4 を使用しているため、AWS で管理している Deep Learning Container(DLC) の PyTorch 1.4 を使いたいところですが、DLC では不足しているライブラリ(libsndfile1)があるため、DLCベースイメージとしてコンテナをカスタマイズします
  * dlc の詳細は[こちら](https://github.com/aws/deep-learning-containers)
  * dlc で使えるコンテナイメージリストは[こちら](https://github.com/aws/deep-learning-containers/blob/master/available_images.md)

### sm-docker のインストール 
* SageMaker Studio では Docker Image のビルドに sm-docker というツールを使うので、最初にインストールします

In [ ]:
!pip install sagemaker-studio-image-build

### Dockerfile の生成
リージョンの情報とアカウント情報が必要なため取得

In [10]:
import boto3
region_name = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity()['Account']

In [ ]:
dockerfile = f"""FROM 763104351884.dkr.ecr.{region_name}.amazonaws.com/pytorch-training:1.4.0-gpu-py3
RUN apt install libsndfile1 -y
"""

with open('./container/Dockerfile','wt') as f:
    f.write(dockerfile)

### Studio アタッチされているロールに以下の信頼関係を追加
以下のコマンドの出力結果をコピーして追加する

In [ ]:
!cat container/trust_relationships.json

### Studio アタッチされているロールにインラインポリシー（もしくはポリシーを別途作成して）アタッチする
以下のコマンドの出力結果をコピーしてアタッチする

In [ ]:
!cat container/inline_policy.json

### コンテナイメージのビルド

In [11]:
IMAGE_NAME = 'sagemaker-pytorch-hifigan'
TAG = ':1.4'
# %cd container
# !sm-docker build . --repository {IMAGE_NAME}{TAG}
# %cd ../

In [12]:
image_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/{IMAGE_NAME}{TAG}'

In [ ]:
!wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2 --no-check-certificate
!tar jxvf  LJSpeech-1.1.tar.bz2 > /dev/null 2>&1

In [2]:
import sagemaker
# input_data_uri = sagemaker.session.Session().upload_data(path='LJSpeech-1.1/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='LJSpeech-1.1')
input_data_uri = 's3://sagemaker-us-east-2-290000338583/LJSpeech-1.1'

In [ ]:
!cp LJSpeech-1.1/wavs/LJ050-0278.wav ./test_wav
!rm -rf LJSpeech-1.1/wavs/
!rm LJSpeech-1.1.tar.bz2

In [ ]:
from sagemaker.pytorch import PyTorch
estimator = PyTorch(
    image_uri=image_uri,
#     py_version='py3', 
#     framework_version='1.4.0',
    entry_point='train.py',
    source_dir = './training_src/',
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'config':'config_v1.json',
        'training_epochs':1
    }
)
estimator.fit(input_data_uri)

2022-03-25 06:52:10 Starting - Starting the training job...
2022-03-25 06:52:25 Starting - Preparing the instances for trainingProfilerReport-1648191129: InProgress
......
2022-03-25 06:53:36 Downloading - Downloading input data...............
2022-03-25 06:56:09 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-03-25 06:56:15,771 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2022-03-25 06:56:15,800 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-03-25 06:56:15,804 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-03-25 06:56:16,106 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2022-03-25 06:56:16,106 sagemaker-containers INFO     Generating setup.cfg
2022-03-25 06:56:16,107 sagemaker-containers IN

In [15]:
job_result = estimator.latest_training_job.describe()

In [16]:
model_s3_uri = job_result['ModelArtifacts']['S3ModelArtifacts']
!aws s3 cp {model_s3_uri} ./

download: s3://sagemaker-us-east-2-290000338583/sagemaker-pytorch-hifigan-2022-03-25-03-34-40-023/output/model.tar.gz to ./model.tar.gz


In [17]:
!tar zxvf model.tar.gz -C ./model

config.json
g_00000809
logs/
logs/events.out.tfevents.1648179549.algo-1.62.0


In [ ]:
!apt update
!apt install libsndfile1 -y
!pip install librosa

In [18]:
%cd inference_src/
!python inference.py --input_wavs_dir ../test_wav --checkpoint_file ../model/g_00000809 --output_dir ../output_wavs_dir
%cd ../

/root/hifi-gan/inference_src
Initializing Inference Process..
Loading '../model/g_00000809'
Complete.
Removing weight norm...
../output_wavs_dir/LJ050-0278_generated.wav
/root/hifi-gan
